In [ ]:
import cv2
import time
from pyzbar import pyzbar
import requests
from bs4 import BeautifulSoup
import os
import re
from urllib.request import urlopen #取得pdf
import requests #取得pdf
import time
from PIL import Image
# |

address = "rtsp://10.X.X.X:X/live"
camera = cv2.VideoCapture(address) #選這項就是用網路相機
#camera = cv2.VideoCapture(0) #選這項就是使用電腦相機
ret, frame = camera.read()
open('drugname.txt', 'w').close()
open('drugreact.txt', 'w').close()
open('NameCn.txt', 'w').close()
open('drugID.txt', 'w').close()
def read_barcodes(frame):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:

        x, y , w, h = barcode.rect
        barcode_text = barcode.data.decode('utf-8')
        cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 255, 0), 2)


        #####爬蟲#####
        response = requests.get(barcode_text)
        soup = BeautifulSoup(response.text, 'lxml')
        result = soup.find('span',{'id':'lblEnName'})
        react = soup.find('span',{'id':'lblIndiCat'}).text
        NameCn = soup.find('span',{'id':'lblChName'}).text
        DrugID = soup.find('span',{'id':'lblLicName'}).text
        DrugID_num = re.findall("\d+", DrugID)[0]
        
        ##這裡下面是抓取pdf檔案的部分
        drag_main_page = urlopen(barcode_text) #获取网页
        drag_main_page_html = BeautifulSoup(drag_main_page, 'html.parser') #解析网页
        drag_main_page_html_str = str(drag_main_page_html)
        drag_pageID_position = str(drag_main_page_html).find('LicId=')
        drag_pageID = drag_main_page_html_str[drag_pageID_position+6:drag_pageID_position+14]

        pdf_url = urlopen('https://info.fda.gov.tw/MLMS/H0001D2.aspx?LicId='+drag_pageID)
        drag_pdf_page_html = BeautifulSoup(pdf_url, 'html.parser') #解析网页

        pdf_page_link = drag_pdf_page_html.find_all('a')  #获取所有超链接
        for h in pdf_page_link:
            pdf_hyperlink = h.get('href')

        r = requests.get('https://info.fda.gov.tw/MLMS/'+pdf_hyperlink)
        pdf = r.content       #響應的二進位制檔案
        with open(drag_pageID+'.pdf','wb') as f:     #二進位制寫入
            f.write(pdf)
        
       
        
        
        
        if result==None:
            print("error")
            data=open("drugname.txt",'w+')
            print("error",file=data)
            data.close()
        else :
            result = soup.find('span',{'id':'lblEnName'}).text

            print(result)
            print(react)
            print(NameCn)
            print(DrugID_num)
            
            
            
            
            data=open("drugname.txt",'w+')
            print(result,file=data)
            data.close()
            data_rec=open("drugreact.txt",'w+')
            print(react,file=data_rec)
            data_rec.close()
            data_Cn=open("NameCn.txt",'w+')
            print(NameCn,file=data_Cn)
            data_Cn.close()
            data_ID=open("drugID.txt",'w+')
            print(DrugID_num,file=data_ID)
            data.close()
            
    return frame
while ret:

    ret, frame = camera.read()
    frame = read_barcodes(frame)
    cv2.imshow('Barcode reader', frame)


    if os.stat("drugID.txt").st_size != 0:
        break
    if cv2.waitKey(1) & 0xFF == 27:
        break

camera.release()
cv2.destroyAllWindows()
